In [1]:
import numpy as np
from scipy.stats import norm
import scipy.integrate as integrate
from scipy.misc import derivative

In [2]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

In [3]:
def g_0(K):
    return K**(1/4)-0.04**(1/2)

def g_1(K):
    return (1/4)*K**(-3/4)

def g_2(K):
    return (-3/16)*K**(-7/4)

In [4]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

In [5]:
def Black76(direction, F, K, sigma, T):
    d1 = (np.log(F/K)+(sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if direction == "call":
        return (F*norm.cdf(d1) - K*norm.cdf(d2))
    if direction == "put":
        return (-F*norm.cdf(-d1) + K*norm.cdf(-d2))

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [6]:
def swaption(direction, S, K, N, sigma, m, T, discount):
    if direction == "pay":  ##pay use call
        #return discount*IRR_0(S, m, N)*Black76("call", S, K, sigma, T)
            return discount*IRR_0(S,m,N)*Black76("call", S, K, sigma, T)
    if direction == "rec": ##receiver use put
        #return discount*IRR_0(S, m, N)*Black76("put", S, K, sigma, T)
        return discount*IRR_0(S,m,N)*Black76("put", S, K, sigma, T)

In [7]:
#sabr sigma changes with k, so need use different sigma over region of integration
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [13]:
# parameters
a = 0.168234
r = -0.358823
n = 0.556587
b = 0.9 
    
F = 0.043612  
     #uncolaterized 0.04}"5117 forward swap  ##colaterized0.043612
N = 10
T = 5
m = 2
discount = 0.98218411973321 #ois discount factor

#### CMS 10y1/p − 0.041/q

In [14]:
def numerical_integration_part_1():
    sigma = SABR(F, F, T, a, b, r, n)
    pay_rec_value = h_1(F, m, N)*(swaption("pay", F, F, N, sigma, m, T, discount) - swaption("rec", F, F, N, sigma, m, T, discount))
    integrand_rec = lambda k: h_2(k, m, N) * swaption("rec", F, k, N, SABR(F, k, T, a, b, r, n), m, T, discount)
    put_part, error = integrate.quad(integrand_rec, 0, F)
    integrand_pay = lambda k: h_2(k, m, N) * swaption("pay", F, k, N, SABR(F, k, T, a, b, r, n), m, T, discount)
    call_part, error = integrate.quad(integrand_pay, F, 5400)
    print(integrand_pay)
    return discount * g_0(F) + put_part+ call_part

In [15]:
payoff_part_1 = numerical_integration_part_1()
payoff_part_1

<function numerical_integration_part_1.<locals>.<lambda> at 0x000002A369E58A60>


<ipython-input-14-80dae3801b70>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  put_part, error = integrate.quad(integrand_rec, 0, F)


0.23694977260270478

#### (CMS 10y1/p − 0.041/q)+

In [16]:
a = 0.168234
r = -0.358823
n = 0.556587
b = 0.9 
    
F = 0.043612  
N = 10
T = 5
m = 2
L = 0.04**(2) 

vol=SABR(F, L, T, a, b, r, n)
pay_rec_value = h_1(0.0016,m,N)*swaption("pay", F, 0.0016, N, vol, m, T, discount)
integrand_pay = lambda k: h_2(k, m, N) * swaption("pay", F, k, N, SABR(F, k, T, a, 0.9, r, n), m, T, discount)
call_part, error = integrate.quad(integrand_pay, 0.0016, 5400)
value = pay_rec_value + call_part

In [17]:
value

0.2591629795667644